In [1]:
import pyspark as ps
import pyspark.sql.functions as f
from pyspark import SQLContext
from pyspark.sql.types import IntegerType, DateType, TimestampType
from datetime import datetime

spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("sparkSQL exercise") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[4] appName=sparkSQL exercise>

In [3]:
sqlContext = SQLContext(sc)

In [4]:
df = sqlContext.read.csv("uk100.csv", header=True)

In [5]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- rank: string (nullable = true)
 |-- last_week_rank: string (nullable = true)
 |-- hmm: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- label: string (nullable = true)
 |-- peak_rank: string (nullable = true)
 |-- weeks_on_chart: string (nullable = true)
 |-- week_of: string (nullable = true)



In [6]:
df.createOrReplaceTempView('test')

In [7]:
result = spark.sql('''SELECT *
            FROM test
            LIMIT 30
            ''')
result.show()

+---+----+--------------+--------+--------------------+--------------------+--------------------+---------+--------------+----------------+
|_c0|rank|last_week_rank|     hmm|               title|              artist|               label|peak_rank|weeks_on_chart|         week_of|
+---+----+--------------+--------+--------------------+--------------------+--------------------+---------+--------------+----------------+
|  0|   1|            11|        |               RIVER|      ELLIE GOULDING|             POLYDOR|        1|             5|December 27 2019|
|  1|   2|             8|        |ALL I WANT FOR CH...|        MARIAH CAREY|            COLUMBIA|        2|            99|December 27 2019|
|  2|   3|             5|        |      LAST CHRISTMAS|                WHAM|                 RCA|        2|            64|December 27 2019|
|  3|   4|            14|        |FAIRYTALE OF NEW ...|POGUES FT KIRSTY ...|         WARNER BROS|        2|            99|December 27 2019|
|  4|   5|          

In [8]:
df = df.drop('_c0')
df = df.drop('hmm')

In [9]:
for col in df.columns:
    df = df.withColumn(col, f.lower(f.col(col)))

In [10]:
df = df.withColumn("rank", df["rank"].cast(IntegerType()))
df = df.withColumn("peak_rank", df["peak_rank"].cast(IntegerType()))
df = df.withColumn("weeks_on_chart", df["weeks_on_chart"].cast(IntegerType()))
#df = df.withColumn("week_of", df["week_of"].cast(DateType()))

In [11]:
def to_date(x):
    return datetime.strptime(x, '%B %d %Y')
hmm = f.udf(lambda y: to_date(y), DateType())


In [12]:
df = df.withColumn("week_of", hmm('week_of'))

In [13]:
df.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- last_week_rank: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- label: string (nullable = true)
 |-- peak_rank: integer (nullable = true)
 |-- weeks_on_chart: integer (nullable = true)
 |-- week_of: date (nullable = true)



In [14]:
df.createOrReplaceTempView('test')

In [15]:
result = spark.sql('''SELECT COUNT(*)
            FROM test
            ''')
result.show()

+--------+
|count(1)|
+--------+
|   26000|
+--------+



In [16]:
result = spark.sql('''SELECT *
            FROM test
            LIMIT 30
            ''')
result.show()

+----+--------------+--------------------+--------------------+--------------------+---------+--------------+----------+
|rank|last_week_rank|               title|              artist|               label|peak_rank|weeks_on_chart|   week_of|
+----+--------------+--------------------+--------------------+--------------------+---------+--------------+----------+
|   1|            11|               river|      ellie goulding|             polydor|        1|             5|2019-12-27|
|   2|             8|all i want for ch...|        mariah carey|            columbia|        2|            99|2019-12-27|
|   3|             5|      last christmas|                wham|                 rca|        2|            64|2019-12-27|
|   4|            14|fairytale of new ...|pogues ft kirsty ...|         warner bros|        2|            99|2019-12-27|
|   5|             2|              own it|stormzy/ed sheera...|      atlantic/merky|        2|             5|2019-12-27|
|   6|            16|merry chris

In [17]:
result = spark.sql('''
            SELECT DISTINCT title, ROUND(AVG(rank), 2)
            FROM test
            WHERE title like "%christmas%"
                OR title like "%merry%"
                OR title like "%xmas%"
                OR title like "%santa%"
                OR title like "%rudolph%"
                OR title like "%reindeer%"
            GROUP BY 1
            ORDER BY 2
            ''')
result.show(50, False)

+----------------------------------------+-----------------------------------+
|title                                   |round(avg(CAST(rank AS BIGINT)), 2)|
+----------------------------------------+-----------------------------------+
|last christmas                          |25.3                               |
|all i want for christmas is you         |26.82                              |
|happy christmas (war is over)           |28.0                               |
|merry christmas everyone                |34.46                              |
|do they know it's christmas             |35.7                               |
|i wish it could be christmas everyday   |36.0                               |
|rockin' around the christmas tree       |37.05                              |
|step into christmas                     |38.84                              |
|driving home for christmas              |39.37                              |
|santa tell me                           |41.0      

In [23]:
result = spark.sql('''
            SELECT COUNT(DISTINCT title)
            FROM test
            ''')
result.show()

+---------------------+
|count(DISTINCT title)|
+---------------------+
|                 2416|
+---------------------+



In [18]:
result = spark.sql('''
            SELECT COUNT(DISTINCT title)
            FROM test
            WHERE rank == 1''')
result.show()

+---------------------+
|count(DISTINCT title)|
+---------------------+
|                   77|
+---------------------+



In [24]:
result = spark.sql('''
            SELECT DISTINCT title, COUNT(*) as num_weeks, MIN(week_of)
            FROM test
            WHERE rank == 1
            GROUP BY 1
            ORDER BY 2 DESC
            ''')
result.show()

+--------------------+---------+------------+
|               title|num_weeks|min(week_of)|
+--------------------+---------+------------+
|           one dance|       15|  2016-04-15|
|        shape of you|       14|  2017-01-13|
|        dance monkey|       11|  2019-10-04|
|   despacito (remix)|       11|  2017-05-12|
|          god's plan|        9|  2018-01-26|
|            rockabye|        9|  2016-11-11|
|            one kiss|        8|  2018-04-20|
|        i don't care|        8|  2019-05-17|
|   someone you loved|        7|  2019-03-01|
|       love yourself|        6|  2015-12-04|
|        thank u next|        6|  2018-11-09|
|             perfect|        6|  2017-12-08|
|            promises|        6|  2018-09-07|
|            senorita|        6|  2019-07-12|
|          cold water|        5|  2016-07-29|
|              havana|        5|  2017-11-03|
|take me back to l...|        5|  2019-08-30|
|             7 years|        5|  2016-02-12|
|    what do you mean|        5|  

In [25]:
result = spark.sql('''
            SELECT DISTINCT title, COUNT(*) as num_weeks, MIN(week_of)
            FROM test
            WHERE peak_rank == 1
            GROUP BY 1
            ORDER BY 2 DESC
            ''')
result.show()

+-------------------+---------+------------+
|              title|num_weeks|min(week_of)|
+-------------------+---------+------------+
|       shape of you|       97|  2017-01-13|
|  thinking out loud|       90|  2015-01-11|
|            perfect|       81|  2017-12-08|
|            shotgun|       74|  2018-06-29|
|        uptown funk|       72|  2015-01-11|
|  despacito (remix)|       71|  2017-05-12|
|              sorry|       65|  2015-11-20|
|          one dance|       64|  2016-04-15|
|   what do you mean|       62|  2015-09-04|
|      love yourself|       62|  2015-12-04|
|          new rules|       61|  2017-08-18|
|               king|       59|  2015-03-08|
|         these days|       59|  2015-01-11|
|love me like you do|       57|  2015-02-08|
|            shallow|       57|  2018-10-26|
|           one kiss|       52|  2018-04-20|
|           stitches|       51|  2016-01-22|
|       hold my hand|       51|  2015-03-29|
|            7 years|       49|  2016-02-12|
|         

In [22]:
result = spark.sql('''
            SELECT AVG(num_weeks)
            FROM (SELECT DISTINCT title, COUNT(*) as num_weeks, MIN(week_of)
                    FROM test
                    WHERE peak_rank == 1
                    GROUP BY 1
                    ORDER BY 2 DESC)
            ''')
result.show()

+------------------+
|    avg(num_weeks)|
+------------------+
|26.708661417322833|
+------------------+



In [21]:
result = spark.sql('''
            SELECT AVG(rank), MAX(weeks_on_chart)
            FROM test
            WHERE title = "shape of you"
            ''')
result.show()

+-----------------+-------------------+
|        avg(rank)|max(weeks_on_chart)|
+-----------------+-------------------+
|42.51546391752577|                 97|
+-----------------+-------------------+



In [22]:
result = spark.sql('''
            SELECT artist, title, COUNT(*)
            FROM test
            GROUP BY artist, title
            ORDER BY 3 DESC
            ''')
result.show()

+--------------------+--------------------+--------+
|              artist|               title|count(1)|
+--------------------+--------------------+--------+
|          ed sheeran|             perfect|     110|
|             killers|       mr brightside|     105|
|          ed sheeran|        shape of you|      97|
|          ed sheeran|   thinking out loud|      90|
|         george ezra|             shotgun|      87|
|                 sia|          chandelier|      87|
|major lazer ft mo...|             lean on|      79|
|           james bay|           let it go|      77|
|settle/greatest s...|          this is me|      75|
|          ed sheeran|          photograph|      73|
|       walk the moon|     shut up & dance|      72|
|mark ronson ft br...|         uptown funk|      72|
|   justin timberlake|can't stop the fe...|      71|
|luis fonsi/daddy ...|   despacito (remix)|      71|
|           james bay| hold back the river|      68|
|       justin bieber|               sorry|   

In [23]:
result = spark.sql('''
            SELECT artist, title, count(*)
            FROM test
            WHERE title = "perfect"
            GROUP BY artist, title
            ''')
result.show()

+-------------+-------+--------+
|       artist|  title|count(1)|
+-------------+-------+--------+
|one direction|perfect|      21|
|   ed sheeran|perfect|     110|
+-------------+-------+--------+



In [36]:
result = spark.sql('''
            SELECT artist, title, label, count(*) as count, ROUND(AVG(rank),2) AS avg_rank
            FROM test
            GROUP BY artist, title, label
            ORDER BY count DESC
            ''')
result.show()

+--------------------+--------------------+--------------------+-----+--------+
|              artist|               title|               label|count|avg_rank|
+--------------------+--------------------+--------------------+-----+--------+
|          ed sheeran|             perfect|              asylum|  110|   48.09|
|             killers|       mr brightside|             mercury|  105|   84.86|
|          ed sheeran|        shape of you|              asylum|   97|   42.52|
|          ed sheeran|   thinking out loud|              asylum|   90|   56.27|
|                 sia|          chandelier|   monkey puzzle/rca|   87|   64.46|
|         george ezra|             shotgun|            columbia|   87|   35.99|
|major lazer ft mo...|             lean on|       because music|   79|   45.92|
|           james bay|           let it go|              virgin|   77|   65.03|
|settle/greatest s...|          this is me|            atlantic|   75|   40.84|
|          ed sheeran|          photogra

In [31]:
result = spark.sql('''
            SELECT label, COUNT(*)
            FROM test
            GROUP BY 1
            ORDER BY 2 DESC
            ''')
result.show()

+--------------------+--------+
|               label|count(1)|
+--------------------+--------+
|            atlantic|    2055|
|            columbia|    1613|
|          interscope|    1493|
|             polydor|    1220|
|                 rca|    1099|
|              virgin|     924|
|              asylum|     911|
|                 emi|     830|
|          syco music|     821|
|    republic records|     817|
|          parlophone|     791|
|         warner bros|     653|
|              island|     650|
|cash money/republ...|     650|
|             capitol|     612|
|             def jam|     519|
|                epic|     471|
|   ministry of sound|     466|
|            positiva|     413|
|          relentless|     360|
+--------------------+--------+
only showing top 20 rows



In [32]:
result = spark.sql('''
            SELECT COUNT(DISTINCT label)
            FROM test
            ''')
result.show()

+---------------------+
|count(DISTINCT label)|
+---------------------+
|                  408|
+---------------------+



In [32]:
result = spark.sql('''
            SELECT label, COUNT(DISTINCT title) AS num_songs, ROUND(AVG(rank),2) AS avg_rank
            FROM test
            GROUP BY 1
            ORDER BY 3
            ''')
result.show()

+--------------------+---------+--------+
|               label|num_songs|avg_rank|
+--------------------+---------+--------+
|        castle music|        1|    13.0|
|            emubands|        1|    15.0|
|      asylum/def jam|        1|   17.14|
|interscope/republ...|        1|   17.56|
|cactus jack/epic/...|        1|    22.6|
|     asylum/columbia|        1|    25.0|
|      emi/syco music|        1|   25.46|
|                  zy|        1|    26.5|
|       columbia/kygo|        1|   26.72|
|black butter/def jam|        2|   26.89|
|       perfect havoc|        1|   27.86|
|def jam/polydor/r...|        1|   27.88|
|             liv'n'g|        1|    28.0|
|polydor/rca/repub...|        1|   28.44|
|  def jam/parlophone|        1|   29.06|
|      merky/atlantic|        1|   29.25|
|        fuller beans|        1|    30.0|
|          dave clark|        1|    31.0|
|   james grant music|        1|    31.0|
|black butter/dave...|        1|    31.2|
+--------------------+---------+--

In [33]:
result = spark.sql('''
            SELECT label, COUNT(DISTINCT title) AS num_songs, ROUND(AVG(rank),2) AS avg_rank
            FROM test
            GROUP BY 1
            ORDER BY 2 DESC
            ''')
result.show()

+--------------------+---------+--------+
|               label|num_songs|avg_rank|
+--------------------+---------+--------+
|          interscope|      171|   51.51|
|                 rca|      136|   53.36|
|            columbia|      134|   48.64|
|            atlantic|      129|   47.02|
|              virgin|      117|   54.65|
|             polydor|      110|   50.31|
|              island|       90|   54.77|
|          syco music|       80|   46.77|
|          parlophone|       79|   51.33|
|cash money/republ...|       78|   50.38|
|                 emi|       76|   47.93|
|             def jam|       69|    54.3|
|         warner bros|       60|   47.58|
|    republic records|       59|   43.67|
|             capitol|       48|   48.49|
|              asylum|       43|   48.22|
|                epic|       42|   53.03|
|   ministry of sound|       34|   46.21|
| republic records/xo|       33|   47.52|
|          relentless|       26|   54.46|
+--------------------+---------+--

In [37]:
result = spark.sql('''
            SELECT MAX(week_of), MIN(week_of)
            FROM test
            ''').show()


+------------+------------+
|max(week_of)|min(week_of)|
+------------+------------+
|  2019-12-27|  2015-01-11|
+------------+------------+



In [39]:
result = spark.sql('''
            SELECT DISTINCT artist, COUNT(*), ROUND(AVG(rank),2) AS avg_rank
            FROM test
            GROUP BY 1
            ORDER BY 2 DESC''').show()

+--------------+--------+--------+
|        artist|count(1)|avg_rank|
+--------------+--------+--------+
|    ed sheeran|     706|   49.83|
|         drake|     352|    49.4|
| justin bieber|     280|   47.48|
| ariana grande|     269|   39.33|
|   george ezra|     263|   48.57|
|           sia|     244|   55.59|
|    little mix|     244|   45.81|
|        weeknd|     228|   46.48|
|     sam smith|     216|   48.84|
|  shawn mendes|     213|   42.92|
|   jess glynne|     207|   44.33|
| years & years|     197|   50.62|
|  taylor swift|     189|   54.21|
|      dua lipa|     172|   41.03|
|     james bay|     169|   58.82|
| lewis capaldi|     158|   29.18|
|      coldplay|     157|   53.01|
|       stormzy|     142|    49.3|
|ellie goulding|     142|   45.52|
|  zara larsson|     142|   43.61|
+--------------+--------+--------+
only showing top 20 rows



In [49]:
result = spark.sql('''
            SELECT title, COUNT(*), MIN(rank) as peak
            FROM test
            WHERE artist = "ed sheeran"
            GROUP BY 1
            ORDER BY 2 DESC
            ''').show(30, False)

+------------------------------+--------+----+
|title                         |count(1)|peak|
+------------------------------+--------+----+
|perfect                       |110     |1   |
|shape of you                  |97      |1   |
|thinking out loud             |90      |3   |
|photograph                    |73      |15  |
|castle on the hill            |53      |2   |
|bloodstream                   |38      |2   |
|galway girl                   |31      |2   |
|sing                          |23      |29  |
|supermarket flowers           |21      |8   |
|don't                         |20      |20  |
|happier                       |18      |6   |
|what do i know                |17      |9   |
|i see fire                    |15      |54  |
|new man                       |14      |5   |
|barcelona                     |13      |12  |
|dive                          |13      |8   |
|how would you feel (paean)    |12      |2   |
|nancy mulligan                |10      |13  |
|hearts don't

In [52]:
result = spark.sql('''
            SELECT title, COUNT(*), MIN(rank) as peak
            FROM test
            WHERE artist = "ed sheeran" AND MIN(rank) < 6
            GROUP BY 1
            ORDER BY 2 DESC
            ''').show()

Py4JJavaError: An error occurred while calling o494.showString.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(title#116, 200)
+- *(1) HashAggregate(keys=[title#116], functions=[partial_count(1), partial_min(rank#170)], output=[title#116, count#945L, min#946])
   +- *(1) Project [cast(lower(rank#11) as int) AS rank#170, lower(title#14) AS title#116]
      +- *(1) Filter ((lower(artist#15) = ed sheeran) && (min(cast(lower(rank#11) as int)) < 6))
         +- *(1) FileScan csv [rank#11,title#14,artist#15] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/work/uk100.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<rank:string,title:string,artist:string>

	at org.apache.spark.sql.catalyst.errors.package$.attachTree(package.scala:56)
	at org.apache.spark.sql.execution.exchange.ShuffleExchangeExec.doExecute(ShuffleExchangeExec.scala:119)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$e

In [48]:
result = spark.sql('''
            SELECT COUNT(DISTINCT title)
            FROM test
            WHERE artist = "ed sheeran"
            
            ''').show()

+---------------------+
|count(DISTINCT title)|
+---------------------+
|                   26|
+---------------------+

